In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from pandas import ExcelWriter
from tqdm import tqdm
import json
import ast
from tqdm import tqdm

def xml2Excel(inDirectory=None, outFileName=None, outDirectory=None):
    path = inDirectory
    rawText1 = open(inDirectory).read()
    rawText = open(inDirectory).read()
    soup = BeautifulSoup(rawText, 'xml')
    rawTexts = []
    tags = []
    original_text = []
    for record in soup.find_all('RECORD'):
        text = record.text
        rawTexts.append(text)
        start = 0
        tag = []
        for phi in record.find_all('PHI'):
            start = text.find(phi.text, start)
            end = start + len(phi.text)
            tagType = phi['TYPE']
            tag.append({'start': str(start), 'end': str(end), 'text': phi.text, 'TYPE': tagType})
        tags.append(tag)
        original_text.append(record)
    print(len(rawTexts), len(tags), len(original_text))
    fullPath = outDirectory + outFileName
    writer = ExcelWriter(outFileName)
    df = pd.DataFrame({'Summaries': rawTexts, 'tags' : tags, 'Annotated_Summaries' : original_text})
    df.to_excel(writer,'Sheet1', index = False)
    writer.close()

xml2Excel(inDirectory='llama_generated_summaries.xml', outDirectory='.',
         outFileName='llama_generated_summaries.xlsx')

dfTrain = pd.read_excel('/content/llama_generated_summaries.xlsx')

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
def func(txt, tgt):
  words = []
  ctag  = []
  j = 0
  for i in tgt:
    k = int(i['start'])
    l = int(i['end'])
    for tt in txt[j:k].strip().split(" "):
      tt = tt.replace("\n", '')
      tt = tt.replace("\n", '').strip()
      if tt != '':
        words.append(tt)
        ctag.append("O")
    info = i["text"].split(" ")
    ctag.append(f"B-{i['TYPE']}")
    for _ in info[1:]:
      ctag.append(f"I-{i['TYPE']}")
    words += [ii.strip() for ii in info]
    j = l
  for tt in txt[j:len(txt)].strip().split(" "):
    tt = tt.replace("\n", '')
    tt = tt.replace("\n", '').strip()
    if tt != '':
      words.append(tt)
      ctag.append("O")
  return words, ctag

In [ ]:
df_train = {'Doc #' : [], 'Words' : [], 'Tags' : [],'segment #' : []}

for i in tqdm(range(dfTrain.shape[0])):
  txt = dfTrain .iloc[i]['Summaries']
  tgt = ast.literal_eval(dfTrain .iloc[i]['tags'])
  a,b = func(txt, tgt)
  df_train["Words"] += a
  df_train["Tags"] += b
  df_train['Doc #'] += [f"Doc-{i}"]* len(a)
  x = int(len(a)/100)
  y = len(a)%100
  for j in range(x):
    df_train['segment #'] += [f"segment {i} {j}"]*100
  df_train['segment #'] += [f"segment {i} {j+1}"]*y
df_train = pd.DataFrame(df_train)

100%|██████████| 1043/1043 [00:01<00:00, 880.76it/s]


In [ ]:
df_train

,Doc #,Words,Tags,segment #
0,Doc-0,\n,O,segment 0 0
1,Doc-0,Max,B-Hospital_Name,segment 0 0
2,Doc-0,Super,I-Hospital_Name,segment 0 0
3,Doc-0,Specialty,I-Hospital_Name,segment 0 0
4,Doc-0,Hospital,I-Hospital_Name,segment 0 0
...,...,...,...,...
429780,Doc-1042,Consultant:,O,segment 1042 3
429781,Doc-1042,Dr.,O,segment 1042 3
429782,Doc-1042,Rohan,O,segment 1042 3
429783,Doc-1042,Singh,O,segment 1042 3


In [ ]:
x=df_train

In [ ]:
# Remove symbols at the start or end of the 'Tokens' column
x['Words'] = x['Words'].str.replace(r'^[\/;,.:\[\]*]+|[\/;,.:\[\]*]+$', '')
x['Words'] = x['Words'].replace('\\n\\n', '')
x['Words'] = x['Words'].replace('\\n\\n\\n', '')
x['Words'] = x['Words'].replace('\\n\\n\\n\\n', '')
x['Words'] = x['Words'].replace('\\n[', '')
x['Words'] = x['Words'].replace('\\n', '')
x['Words'] = x['Words'].replace('\\n[', '')
x['Words'] = x['Words'].str.replace(r'^[\/;,.:\[\]*\\n**]+|[\/;,.:\[\]*\\n**]+$', '')
x['Words'] = x['Words'].str.replace(r'^[\/;,.:\[\]*]+|[\/;,.:\[\]*]+$', '')

In [ ]:
x= x.dropna()

In [ ]:
x['Tags'].value_counts()

Tags
O                                 384095
B-Treatment_Date                    5344
B-Treatment_Time                    4532
I-Doctor_Name                       3262
B-Ward_Location                     2474
                                   ...  
B-Diagnosis / Chief Complaints         1
B-Admission Details                    1
I-Family_Member_Name                   1
B-Family_Member_Name                   1
B-Email_ID                             1
Name: count, Length: 178, dtype: int64

In [ ]:
x.to_csv("llama_generated_summary_with_original_tags.csv")

In [ ]:
x=pd.read_csv('/content/llama_generated_summary_with_original_tags.csv')

#Tag mapping with the tag set of n2c2 2006 dataset

In [ ]:
def update_tags(df):
    df['Tags'][df['Tags'] == 'B-Treatment_Date'] = 'B-DATE'
    df['Tags'][df['Tags'] == 'I-Treatment_Time'] = 'O'
    df['Tags'][df['Tags'] == 'B-Treatment_Time'] = 'O'
    df['Tags'][df['Tags'] == 'B-Ward_Location'] = 'B-LOCATION'
    df['Tags'][df['Tags'] == 'B-Patient_ID'] = 'B-ID'
    df['Tags'][df['Tags'] == 'I-Treatment_Date'] = 'I-DATE'
    df['Tags'][df['Tags'] == 'I-Age'] = 'B-AGE'
    df['Tags'][df['Tags'] == 'B-Gender'] = 'O'
    df['Tags'][df['Tags'] == 'I-Ward_Location'] = 'I-HOSPITAL'
    df['Tags'][df['Tags'] == 'I-Doctor_Name'] = 'I-DOCTOR'
    df['Tags'][df['Tags'] == 'B-Patient_Name'] = 'B-PATIENT'
    df['Tags'][df['Tags'] == 'B-Age'] = 'B-AGE'
    df['Tags'][df['Tags'] == 'I-Patient_Name'] = 'I-PATIENT'
    df['Tags'][df['Tags'] == 'B-Doctor_Name'] = 'B-DOCTOR'
    df['Tags'][df['Tags'] == 'I-Hospital_Name'] = 'I-HOSPITAL'
    df['Tags'][df['Tags'] == 'B-Zip'] = 'B-PHONE'
    df['Tags'][df['Tags'] == 'B-Other_Info'] = 'O'
    df['Tags'][df['Tags'] == 'B-City'] = 'B-LOCATION'
    df['Tags'][df['Tags'] == 'B-Hospital_Name'] = 'B-HOSPITAL'
    df['Tags'][df['Tags'] == 'B-State'] = 'B-LOCATION'
    df['Tags'][df['Tags'] == 'B-Country'] = 'B-LOCATION'
    df['Tags'][df['Tags'] == 'I-Street'] = 'I-LOCATION'
    df['Tags'][df['Tags'] == 'B-Staff_Name'] = 'B-DOCTOR'
    df['Tags'][df['Tags'] == 'I-Staff_Name'] = 'I-DOCTOR'
    df['Tags'][df['Tags'] == 'I-State'] = 'I-LOCATION'
    df['Tags'][df['Tags'] == 'B-Phone_No'] = 'B-PHONE'
    df['Tags'][df['Tags'] == 'B-Doctor_Specialisation'] = 'O'
    df['Tags'][df['Tags'] == 'B-Landline'] = 'B-PHONE'
    df['Tags'][df['Tags'] == 'I-Patient_Profession'] = 'O'
    df['Tags'][df['Tags'] == 'B-IP_Address'] = 'B-PHONE'
    df['Tags'][df['Tags'] == 'B-Patient_Profession'] = 'O'
    df['Tags'][df['Tags'] == 'I-Doctor_Specialisation'] = 'O'
    df['Tags'][df['Tags'] == 'B-Misc_Medical_ID'] = 'B-ID'
    df['Tags'][df['Tags'] == 'I-Gaurdian_Name'] = 'I-PATIENT'
    df['Tags'][df['Tags'] == 'B-Gaurdian_Name'] = 'B-PATIENT'
    df['Tags'][df['Tags'] == 'B-Patient_DOB'] = 'B-DATE'
    df['Tags'][df['Tags'] == 'I-City'] = 'I-LOCATION'
    df['Tags'][df['Tags'] == 'I-Other_Info'] = 'O'
    df['Tags'][df['Tags'] == 'I-Gender'] = 'O'
    df['Tags'][df['Tags'] == 'I-Patient_DOB'] = 'I-DATE'
    df['Tags'][df['Tags']=='B-Other_Location'] = 'B-LOCATION'
    df['Tags'][df['Tags']=='I-Other_Location'] = 'I-LOCATION'
    df['Tags'][df['Tags']=='B-Street'] = 'B-LOCATION'
    df['Tags'][df['Tags']=='I-Country'] = 'I-LOCATION'
    df['Tags'][df['Tags'] == 'I-TIME'] = 'O'
    df['Tags'][df['Tags'] == 'B-TIME'] = 'O'
    df['Tags'][df['Tags'] == 'I-Correspondence_Address'] = 'I-LOCATION'
    df['Tags'][df['Tags'] == 'I-Address'] = 'I-LOCATION'
    df['Tags'][df['Tags'] == 'I-Investigations'] = 'O'
    df['Tags'][df['Tags'] == 'I-Investigation'] = 'O'
    df['Tags'][df['Tags'] == 'I-Medications'] = 'O'
    df['Tags'][df['Tags'] == 'I-Systemic_Examination'] = 'O'
    df['Tags'][df['Tags'] == 'I-Physical Examination'] = 'O'
    df['Tags'][df['Tags'] == 'I-Procedure_Result'] = 'O'
    df['Tags'][df['Tags'] == 'I-Admission Details'] = 'O'
    df['Tags'][df['Tags'] == 'I-Investigation_Result'] = 'O'
    df['Tags'][df['Tags'] == 'I-Special Instructions'] = 'O'
    df['Tags'][df['Tags'] == 'I-Follow-up Instructions'] = 'O'
    df['Tags'][df['Tags'] == 'I-Medical_History'] = 'O'
    df['Tags'][df['Tags'] == 'I-Diagnosis'] = 'O'
    df['Tags'][df['Tags'] == 'B-Phone'] = 'B-PHONE'
    df['Tags'][df['Tags'] == 'B-Address'] = 'B-LOCATION'
    df['Tags'][df['Tags'] == 'I-Medication'] = 'O'
    df['Tags'][df['Tags'] == 'I-Special_Instructions'] = 'O'
    df['Tags'][df['Tags'] == 'I-Treatment Plan'] = 'O'
    df['Tags'][df['Tags'] == 'I-Medical History'] = 'O'
    df['Tags'][df['Tags'] == 'I-Diagnosis / Chief Complaints'] = 'O'
    df['Tags'][df['Tags'] == 'I-Procedures/Lab Tests Conducted'] = 'O'
    df['Tags'][df['Tags'] == 'B-Investigation'] = 'O'
    df['Tags'][df['Tags'] == 'I-Physical_Examination'] = 'O'
    df['Tags'][df['Tags'] == 'I-Doctor_Signature'] = 'I-DOCTOR'
    df['Tags'][df['Tags'] == 'I-Family_Medical_History'] = 'O'
    df['Tags'][df['Tags'] == 'I-Procedures/Lab_Tests_Conducted'] = 'O'
    df['Tags'][df['Tags'] == 'I-Phone_No'] = 'I-PHONE'
    df['Tags'][df['Tags'] == 'I-Procedure'] = 'O'
    df['Tags'][df['Tags'] == 'B-Contact_Info'] = 'B-PHONE'
    df['Tags'][df['Tags'] == 'I-Family Medical history'] = 'O'
    df['Tags'][df['Tags'] == 'B-Email'] = 'B-PHONE'
    df['Tags'][df['Tags'] == 'B-Medication'] = 'O'
    df['Tags'][df['Tags'] == 'B-Doctor_Signature'] = 'B-DOCTOR'
    df['Tags'][df['Tags'] == 'I-Contact_Address'] = 'B-LOCATION'
    df['Tags'][df['Tags'] == 'I-Family_Info'] = 'O'
    df['Tags'][df['Tags'] == 'B-Medication_Name'] = 'O'
    df['Tags'][df['Tags'] == 'B-Phone_Number'] = 'B-PHONE'
    df['Tags'][df['Tags'] == 'B-Procedure'] = 'O'
    df['Tags'][df['Tags'] == 'I-Medication_Name'] = 'O'
    df['Tags'][df['Tags'] == 'I-Procedure/Lab Test'] = 'O'
    df['Tags'][df['Tags'] == 'I-Phone'] = 'I-PHONE'
    df['Tags'][df['Tags'] == 'I-Signature of Consultant'] = 'I-DOCTOR'
    df['Tags'][df['Tags'] == 'B-Diagnosis'] = 'O'
    df['Tags'][df['Tags'] == 'B-Investigation_Date'] = 'B-DATE'
    df['Tags'][df['Tags'] == 'B-Phone No'] = 'B-PHONE'
    df['Tags'][df['Tags'] == 'B-Investigation_Time'] = 'O'
    df['Tags'][df['Tags'] == 'I-Contact_Info'] = 'I-PHONE'
    df['Tags'][df['Tags'] == 'I-Phone_Number'] = 'I-PHONE'
    df['Tags'][df['Tags'] == 'B-Contact_No'] = 'B-PHONE'
    df['Tags'][df['Tags'] == 'B-Patient_Type'] = 'O'
    df['Tags'][df['Tags'] == 'I-Prepared by'] = 'I-DOCTOR'
    df['Tags'][df['Tags'] == 'I-Contact Information'] = 'I-PHONE'
    df['Tags'][df['Tags'] == 'I-Contact_Number'] = 'I-PHONE'
    df['Tags'][df['Tags'] == 'B-Procedure/Lab Test'] = 'O'
    df['Tags'][df['Tags'] == 'B-Contact_Address'] = 'B-LOCATION'
    df['Tags'][df['Tags'] == 'B-Signature'] = 'B-DOCTOR'
    df['Tags'][df['Tags'] == 'B-Mobile'] = 'B-PHONE'
    df['Tags'][df['Tags'] == 'B-Phone Number'] = 'B-PHONE'
    df['Tags'][df['Tags'] == 'I-Allergy'] = 'O'
    df['Tags'][df['Tags'] == 'B-Date'] = 'B-DATE'
    df['Tags'][df['Tags'] == 'B-Investigations'] = 'O'
    df['Tags'][df['Tags'] == 'B-Allergies'] ='O'
    df['Tags'][df['Tags'] == 'B-Admission_No'] = 'O'
    df['Tags'][df['Tags'] == 'B-Contact'] = 'B-PHONE'
    df['Tags'][df['Tags'] == 'B-Procedure_Result'] = 'O'
    df['Tags'][df['Tags'] == 'B-Procedure_Date'] = 'B-DATE'
    df['Tags'][df['Tags'] == 'I-Patient_Signature'] = 'I-PATIENT'
    df['Tags'][df['Tags'] == 'I-Procedure_Name'] = 'O'
    df['Tags'][df['Tags'] == 'B-Procedure_Name'] = 'O'
    df['Tags'][df['Tags'] == 'B-Pin'] = 'B-LOCATION'
    df['Tags'][df['Tags'] == 'B-Allergy'] = 'O'
    df['Tags'][df['Tags'] == 'B-Follow_up_Date'] = 'O'
    df['Tags'][df['Tags'] == 'B-Department'] = 'B-HOSPITAL'
    df['Tags'][df['Tags'] == 'I-Department'] = 'I-HOSPITAL'
    df['Tags'][df['Tags'] == 'I-Signature'] = 'I-DOCTOR'
    df['Tags'][df['Tags'] == 'I-Disease'] = 'O'
    df['Tags'][df['Tags'] == 'B-Temperature'] = 'O'
    df['Tags'][df['Tags'] == 'B-Patient_Email'] = 'B-PHONE'
    df['Tags'][df['Tags'] == 'B-Contact Number'] = 'B-PHONE'
    df['Tags'][df['Tags'] == 'B-Follow-up Instructions'] = 'O'
    df['Tags'][df['Tags'] == 'B-Procedures/Lab Tests Conducted'] = 'O'
    df['Tags'][df['Tags'] == 'B-Special Instructions'] = 'O'
    df['Tags'][df['Tags'] == 'B-Signature of Consultant'] = 'B-DOCTOR'
    df['Tags'][df['Tags'] == 'B-Prepared by'] = 'B-DOCTOR'
    df['Tags'][df['Tags'] == 'B-Investigation_Name'] = 'O'
    df['Tags'][df['Tags'] == 'I-Patient_ID'] = 'I-ID'
    df['Tags'][df['Tags'] == 'I-Investigation_Name'] = 'O'
    df['Tags'][df['Tags'] == 'B-Investigation_Result'] = 'O'
    df['Tags'][df['Tags'] == 'B-Family_Info'] = 'O'
    df['Tags'][df['Tags'] == 'I-Patient_Type'] = 'O'
    df['Tags'][df['Tags'] == 'I-Zip'] = 'I-LOCATION'
    df['Tags'][df['Tags'] == 'B-Correspondence_Address'] = 'B-LOCATION'
    df['Tags'][df['Tags'] == 'B-Name'] = 'B-PATIENT'
    df['Tags'][df['Tags'] == 'B-Contact Information'] = 'B-LOCATION'
    df['Tags'][df['Tags'] == 'B-Admission Date'] = 'B-DATE'
    df['Tags'][df['Tags'] == 'B-Admission Time'] = 'O'
    df['Tags'][df['Tags'] == 'B-Admission Number'] = 'B-ID'
    df['Tags'][df['Tags'] == 'B-Staff Name'] = 'B-DOCTOR'
    df['Tags'][df['Tags'] == 'I-State'] = 'I-LOCATION'
    df['Tags'][df['Tags'] == 'I-Staff Name'] = 'I-DOCTOR'
    df['Tags'][df['Tags'] == 'B-Disease'] = 'O'
    df['Tags'][df['Tags'] == 'B-Treatment Plan'] = 'O'
    df['Tags'][df['Tags'] == 'B-Patient_Signature'] = 'B-PATIENT'
    df['Tags'][df['Tags'] == 'B-Patient_Phone'] = 'B-PHONE'
    df['Tags'][df['Tags'] == 'B-Medications'] = 'O'
    df['Tags'][df['Tags'] == 'B-Pin Code'] = 'B-LOCATION'
    df['Tags'][df['Tags'] == 'B-Family Medical history'] = 'O'
    df['Tags'][df['Tags'] == 'B-Medical History'] = 'O'
    df['Tags'][df['Tags'] == 'B-Contact_Number'] = 'B-PHONE'
    df['Tags'][df['Tags'] == 'B-Contact_Phone'] = 'B-PHONE'
    df['Tags'][df['Tags'] == 'I-Phone No'] = 'I-PHONE'
    df['Tags'][df['Tags'] == 'B-Pin_No'] = 'B-LOCATION'
    df['Tags'][df['Tags'] == 'B-Systemic_Examination'] = 'O'
    df['Tags'][df['Tags'] == 'B-Patient_Spouse'] = 'B-PATIENT'
    df['Tags'][df['Tags']=='I-Patient_Spouse'] = 'I-PATIENT'
    df['Tags'][df['Tags'] == 'B-Physical Examination'] = 'O'
    df['Tags'][df['Tags'] == 'B-Special_Instructions'] = 'O'
    df['Tags'][df['Tags'] == 'B-Diagnosis / Chief Complaints'] = 'O'
    df['Tags'][df['Tags'] == 'B-Admission Details'] = 'O'
    df['Tags'][df['Tags'] == 'I-Family_Member_Name'] = 'I-PATIENT'
    df['Tags'][df['Tags'] == 'B-Family_Member_Name'] = 'B-PATIENT'
    df['Tags'][df['Tags'] == 'B-Employee_ID'] = 'B-ID'
    df['Tags'][df['Tags'] == 'B-Postal_Code'] = 'B-LOACTION'
    df['Tags'][df['Tags'] == 'B-Weight'] = 'O'
    df['Tags'][df['Tags'] == 'B-Procedures/Lab_Tests_Conducted'] = 'O'
    df['Tags'][df['Tags'] == 'B-Physical_Examination'] = 'O'
    df['Tags'][df['Tags'] == 'B-Family_Medical_History'] = 'O'
    df['Tags'][df['Tags'] == 'B-Medical_History'] = 'O'
    df['Tags'][df['Tags'] == 'B-Chest'] = 'O'
    df['Tags'][df['Tags'] == 'B-BMI'] = 'O'
    df['Tags'][df['Tags'] == 'I-Weight'] = 'O'
    df['Tags'][df['Tags'] == 'I-Height'] = 'O'
    df['Tags'][df['Tags'] == 'B-Height'] = 'O'
    df['Tags'][df['Tags'] == 'B-Email_ID'] = 'B-PHONE'
    return df

In [ ]:
xy = update_tags(x)

<ipython-input-78-e187f224c081>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Tags'][df['Tags'] == 'B-Treatment_Date'] = 'B-DATE'
<ipython-input-78-e187f224c081>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Tags'][df['Tags'] == 'I-Treatment_Time'] = 'O'
<ipython-input-78-e187f224c081>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Tags'][df['Tags'] == 'B-Treatment_Time'] = 'O'
<ipython-input-78-e187f224c081>:5: Set

In [ ]:
xy['Tags'].value_counts()

Tags
O             394126
B-DATE          5369
B-LOCATION      4819
I-DOCTOR        3883
I-LOCATION      3611
B-HOSPITAL      3450
I-HOSPITAL      3321
B-ID            2259
B-DOCTOR        2044
B-PHONE         2010
I-PATIENT       1287
B-AGE           1254
B-PATIENT       1243
I-AGE           1010
I-DATE            68
I-PHONE           31
Name: count, dtype: int64

In [ ]:
xy.to_csv("llama_generated_summary_tag_mapped_to_n2c2_2006.csv")